In [ ]:
import ast
import pickle as pkl

from plot_utils import (
    plot_camera_locations_distribution,
)
from data_utils import (
    results2df,
    return_ct_location_segments,
)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

**Load annotations and results**

In [ ]:
# Relevant paths
train_results_file = (
    "../dataset/results/model=slow_r50-w-negatives_e=100_split=train.pkl"
)
val_results_file = "../dataset/results/model=slow_r50-w-negatives_e=100_split=val.pkl"


model_name = "slow_r50-w-negatives"
folder_path = "../dataset/results/"
metadata_file = "../dataset/metadata/metadata.csv"
behavioural_labels_file = "../dataset/metadata/behaviours.txt"
segements_file = "../dataset/metadata/segments.txt"

In [ ]:
with open(
    train_results_file,
    "rb",
) as f:
    train_data = pkl.load(f)

with open(val_results_file, "rb") as f:
    val_data = pkl.load(f)

metadata_df = pd.read_csv(metadata_file)

with open(behavioural_labels_file, "rb") as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

with open(segements_file, "rb") as f:
    segments = [seg.decode("utf-8").strip() for seg in f.readlines()]

In [ ]:
# This will return dfs with preds, feats, labels, and all metadata
train_df, val_df = results2df(train_data, val_data, metadata_df)

In [ ]:
# Do this to inspect camera location segments
h_df, t_df, f_df = return_ct_location_segments(train_df, head=50, tail=10)

# Merge with original dataframe
h_df = h_df.merge(train_df, on="utm", how="left")
t_df = t_df.merge(train_df, on="utm", how="left")
f_df = f_df.merge(train_df, on="utm", how="left")

In [ ]:
# Usage:
plot_camera_locations_distribution(train_df, head=50, tail=10, use_proportion=True)